<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Model_based_CartPole_250320_V3_Both_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.6MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 460kB 8.8MB/s 
     |████████████████████████████████| 112kB 16.5MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=5895068459ed47662bcccfa9bb49fd266e2f21c19789a3c2de3d16a601514eba
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=be7976fb63e63def67bfa3cf44d7fbf579391f54054b69e862ced09308a9d0a2
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=10c7d

In [0]:
from __future__ import print_function
import pickle
import sys

import argparse

import random
import torch
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import RandomSampler, BatchSampler
from torchvision import transforms
import torchvision.transforms.functional as TF

from tqdm import tqdm
import datetime
import time
import gym

from torch.autograd import Variable
import wandb
#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Imported required for the Model-based RL
from sklearn.preprocessing import StandardScaler

In [3]:
!wandb login 11d787a211e05ca01c50131c5724e375cd5d3364

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
class EarlyStopping(object):
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=5, verbose=False, wandb=None, name=""):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = 1e11 # Nawid - Set a very high initial best loss
        self.name = name
        self.wandb = wandb

    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score >= self.best_score: # Nawid - Inverse signs to take into minimising loss instead of maximising accuracy
            self.counter += 1
            print(f'EarlyStopping for {self.name} counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                print(f'{self.name} has stopped')
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(
                f'Validation loss decreased/improved for {self.name}  ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')

        save_dir = self.wandb.run.dir
        torch.save(model.state_dict(), save_dir + "/" + self.name + ".pt")
        self.wandb.save(save_dir + "/" + self.name + ".pt")
        self.val_loss_min = val_loss


# Oracle models

In [0]:
from gym.envs.classic_control import CartPoleEnv
import math
from gym import spaces, logger
from gym.utils import seeding

class CartPoleTask(CartPoleEnv):
    def predict_reward(self, state, action, next_state):
        x, theta = next_state[:, 0], next_state[:, 2]
        done = ((x < -self.x_threshold) \
                | (x > self.x_threshold) \
                | (theta < -self.theta_threshold_radians) \
                | (theta > self.theta_threshold_radians))#.float()
        cost = done - 1.0
        reward = - cost # Nawid- Finds the reward
        
        reward = np.expand_dims(reward,axis=1)
        return reward
    
    def get_reset_state(self, n): # Nawid - Resets the state
        return torch.FloatTensor(n, 4).uniform_(-0.05, 0.05)

    def set_new_state(self, state): # Nawid - Used to manually change the state if necessary
        self.state = state 

    def step(self, action, *args, **kwargs):
        next_state, reward, done, info = super().step(action, *args, **kwargs)
        if done:
            reward = 0
        return np.float32(next_state), reward, done, info
    
    def predict_states(self, states, actions): # Nawid - Obtains the next state, this is obtained from the step function
        x, x_dot, theta, theta_dot = states[:,0], states[:,1],states[:,2],states[:,3]
        force = np.array([self.force_mag if action == 1 else -self.force_mag for action in actions])
        #force = self.force_mag if action==1 else -self.force_mag
        cos_vectorised = np.vectorize(math.cos) # Nawid-  Used to vectorise the math.cos function which usually takes one input
        sin_vectorised = np.vectorize(math.sin)
        costheta = cos_vectorised(theta)
        sintheta = sin_vectorised(theta)
        #costheta = math.cos(theta)
        #sintheta = math.sin(theta)
        
        temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta* temp) / (self.length * (4.0/3.0 - self.masspole * costheta * costheta / self.total_mass))
        xacc  = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        if self.kinematics_integrator == 'euler':
            x  = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else: # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x  = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot
        
        
        next_states = np.array((x,x_dot,theta,theta_dot)) # Nawid - Joins together the different (200,) arrays to get a (4,200) 
        next_states = np.transpose(next_states) # Nawid - Outputs 200,4 shape
        
        return next_states

    def eval(self): # Nawid - Does nothing but is a function which is consistent with neural network models
        pass
    
    def train(self): # Nawid - Does nothing but is a function which is consistent with the neural network models
        pass


In [0]:
def one_hot(i):
    a = np.zeros(n_actions, 'uint8')
    a[i] = 1
    return a

# Neural Network models

In [0]:
class NNDynamicsModel(nn.Module):
    '''
    Model that predict the next state, given the current state and action
    '''
    def __init__(self, input_dim, obs_output_dim):
        super(NNDynamicsModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, obs_output_dim)
        )
    def forward(self, x):
        return self.mlp(x.float())

class NNRewardModel(nn.Module):
    '''
    Model that predict the reward given the current state and action
    '''
    def __init__(self, input_dim, reward_output_dim):
        super(NNRewardModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, reward_output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x.float())

def model_MSEloss(y_pred,y_truth, device):
    '''
    Compute the MSE (Mean Squared Error)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.mse_loss(y_pred.view(-1).float(), y_truth.view(-1))

def model_CEloss(y_pred,y_truth,device):
    '''
    Compute the CEloss
    '''
    y_truth = torch.Tensor(np.array(y_truth)).to(device)
    return F.cross_entropy(y_pred, y_truth)

def model_BCEloss(y_pred,y_truth,device):
    '''
    Compute the BCE (Binary cross entropy)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.binary_cross_entropy(y_pred.view(-1).float(), y_truth.view(-1))

# Data collection and preprocessing

In [0]:
def gather_random_trajectories(num_traj,env_name):
    '''
    Run num_traj random trajectories to gather information about the next state and reward.
    Data used to train the models in a supervised way.
    '''
    dataset_random = []
    env = gym.make(env_name)
    game_rewards = []
    for n in range(num_traj):

        obs = env.reset()
        while True:
            sampled_action = env.action_space.sample()
            sampled_action_one_hot = one_hot(sampled_action)
            new_obs, reward, done, _ = env.step(sampled_action)
            

            dataset_random.append([obs, new_obs, reward, done, sampled_action_one_hot])

            obs = new_obs
            game_rewards.append(reward)

            if done:
                break

    # print some stats
    print('Mean R:',np.round(np.sum(game_rewards)/num_traj,2), 'Max R:', np.round(np.max(game_rewards),2), np.round(len(game_rewards)/num_traj))

    return dataset_random

In [0]:
def collate_data(random_dataset, rl_dataset,num_examples_added):
    if len(rl_dataset) > 0:
        '''
        # To use only a fraction of the random dataset
        rand = np.arange(len(random_dataset))
        np.random.shuffle(rand)
        rand = rand[:int(len(rl_dataset)*0.8)] # 80% of rl dataset
        d_concat = np.concatenate([np.array(random_dataset)[rand], rl_dataset], axis=0)'''        
        random_dataset = np.array(random_dataset)
        rl_dataset = np.array(rl_dataset)
        print('random_dataset', random_dataset.shape)
        print('rl dataset', rl_dataset.shape)
        
        d_concat = np.concatenate([random_dataset, rl_dataset], axis=0)
        print('d_concat',d_concat.shape)        
    else:
        d_concat = np.array(random_dataset)
    
    # Split the dataset into train(80%) and test(20%)
    D_train = d_concat[:int(-num_examples_added*1/5)]
    D_valid = d_concat[int(-num_examples_added*1/5):]

    print("len(D):", len(d_concat), 'len(Dtrain)', len(D_train))

    # Shuffle the dataset
    sff = np.arange(len(D_train))
    np.random.shuffle(sff)
    D_train = D_train[sff]

    # Create the input and output for the train
    X_env_train = np.array([np.concatenate([obs,act]) for obs,_,_,_,act in D_train]) # Takes obs and action
    # Env output
    y_env_train = np.array([no for _,no,_,_,_ in D_train])
    y_env_train = y_env_train - np.array([obs for obs,_,_,_,_ in D_train]) # y(state) = s(t+1) - s(t)

    # Reward's output
    
    X_rew_train = np.array([np.concatenate([obs,act,no]) for obs,no,_,_,act in D_train]) # Takes obs and action and next state
    y_rew_train = np.array([[D] for _,_,_,D,_ in D_train])
    #y_rew_train = np.array([[rw] for _,_,rw,_,_ in D_train])
    
    # Create the input and output array for the validation
    X_env_val = np.array([np.concatenate([obs,act]) for obs,_,_,_,act in D_valid]) # Takes obs and action
    X_rew_val = np.array([np.concatenate([obs,act,no]) for obs,no,_,_,act in D_valid]) # Takes obs and action
    
    # Reward output
    y_rew_val = np.array([[D] for _,_,_,D,_ in D_valid])
    #y_rew_val = np.array([[rw] for _,_,rw,_,_ in D_valid])

    # Next state output
    y_env_val = np.array([no for _,no,_,_,_ in D_valid])
    y_env_val = y_env_val - np.array([obs for obs,_,_,_,_ in D_valid]) # y(state) = s(t+1) - s(t)

    env_train_data, env_val_data = (X_env_train, y_env_train), (X_env_val, y_env_val) 
    rew_train_data, rew_val_data = (X_rew_train, y_rew_train), (X_rew_val, y_rew_val) 

    return env_train_data, env_val_data, rew_train_data, rew_val_data 

In [0]:
def normalise_data(env_train, env_val, rew_train,rew_val):
    (X_env_train, y_env_train), (X_env_val, y_env_val) =  env_train, env_val
    (X_rew_train, y_rew_train), (X_rew_val, y_rew_val) =  rew_train, rew_val
    env_input_scaler = StandardScaler()
    
    # Standard the input features by removign the mean and scaling to unit variance
    X_env_train = env_input_scaler.fit_transform(X_env_train)
    X_env_val = env_input_scaler.transform(X_env_val)

    # Standardize the outputs by removing the mean and scaling to unit variance
    env_output_scaler = StandardScaler()
    y_env_train = env_output_scaler.fit_transform(y_env_train)
    y_env_val = env_output_scaler.transform(y_env_val)

    # Standardise input to reward model
    rew_input_scaler = StandardScaler()
    X_rew_train = rew_input_scaler.fit_transform(X_rew_train)
    X_rew_train = rew_input_scaler.transform(X_rew_train)

    # Standardise output of reward model
    rew_output_scaler = StandardScaler()
    y_rew_train = rew_output_scaler.fit_transform(y_rew_train)
    y_rew_val = rew_output_scaler.transform(y_rew_val)
    norm = (env_input_scaler, env_output_scaler, rew_input_scaler, rew_output_scaler)

    norm_env_train_data, norm_env_val_data = (X_env_train, y_env_train), (X_env_val, y_env_val) 
    norm_rew_train_data, norm_rew_val_data = (X_rew_train, y_rew_train), (X_rew_val, y_rew_val)

    return norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, norm

In [0]:
def normalise_v2(env_train,env_val,rew_train,rew_val,norm):
    (X_env_train, y_env_train), (X_env_val, y_env_val) =  env_train, env_val
    (X_rew_train, y_rew_train), (X_rew_val, y_rew_val) =  rew_train, rew_val
    
    env_input_scaler, env_output_scaler, rew_input_scaler, rew_output_scaler =norm

    X_env_train = env_input_scaler.transform(X_env_train)
    X_env_val = env_input_scaler.transform(X_env_val)

    y_env_train = env_output_scaler.transform(y_env_train)
    y_env_val = env_output_scaler.transform(y_env_val)

    # Standardise input to reward model
    
    X_rew_train = rew_input_scaler.transform(X_rew_train)
    X_rew_train = rew_input_scaler.transform(X_rew_train)

    # Standardise output of reward model
    y_rew_train = rew_output_scaler.transform(y_rew_train)
    y_rew_val = rew_output_scaler.transform(y_rew_val)

    norm_env_train_data, norm_env_val_data = (X_env_train, y_env_train), (X_env_val, y_env_val) 
    norm_rew_train_data, norm_rew_val_data = (X_rew_train, y_rew_train), (X_rew_val, y_rew_val)

    return norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data

# Model training

In [0]:
def train_model(train_data,val_data,model,batch_size, max_model_iter, optimizer, device,early_stopper,desired_model='Env_model'):
    ''' 
    General function to train either of the two models
    '''

    (X_train, y_train), (X_val, y_val) =  train_data, val_data
    losses_env = []
    if desired_model =='Env_model': # Nawid -  Decides which loss function to use
        loss_function = model_MSEloss
    else:
        loss_function = model_BCEloss

    # go through max_model iter supervised iterations
    for it in tqdm(range(max_model_iter)):
        # create mini batches of size batch_size
        for mb in range(0,len(X_train), batch_size): # Nawid- Batch size is the step size
            if len(X_train) > mb + BATCH_SIZE:
                X_mb = X_train[mb:mb+BATCH_SIZE]
                y_mb = y_train[mb:mb+BATCH_SIZE]
                #X_mb += np.random.normal(loc = 0, scale = 0.001, size= X_mb.shape)

                optimizer.zero_grad()
                # forward pass of model to compute the output
                pred_mb = model(torch.tensor(X_mb).to(device))
                

                # compute the MSE loss
                
                loss = loss_function(pred_mb,y_mb,device) #Nawid-  Uses Mse loss if dynamics model or uses BCE loss if reward/done model
                wandb.log({'{} Training loss'.format(desired_model):loss.cpu().detach().numpy()})
                # backward pass
                loss.backward()
                # optimization step
                optimizer.step()
        
        # Nawid - Calculate the validation loss after each epoch
        model.eval()
        with torch.no_grad():
            pred_val = model(torch.tensor(X_val).to(device))
            val_loss = loss_function(pred_val, y_val,device)
            wandb.log({'{} Validation loss'.format(desired_model):val_loss})
        model.train()

        early_stopper(val_loss,model)
        if early_stopper.early_stop:
            print("Early stopping")
            break

# Controller

In [0]:
def multi_model_based_control(env_model, rew_model, real_obs, num_sequences, horizon_length, sample_action, norm,device):
    '''
    Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
    '''
    best_reward = -1e9
    best_next_action = []
    
    (env_input_scaler, env_output_scaler, rew_input_scaler, rew_output_scaler) =norm
    m_obs = np.array([real_obs for _ in range(num_sequences)])
    
    # array that contains the rewards for all the sequence
    unroll_rewards = np.zeros((num_sequences, 1)) 
    first_sampled_actions = []

    env_model.eval()
    rew_model.eval()

    # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
    ## i.e roll a given number of trajectories in a single batch (to increase speed)
    
    for t in range(horizon_length):
        # sample actions for each sequence
        sampled_actions = np.array([sample_action() for _ in range(num_sequences)])
        sampled_actions_one_hot = np.array([one_hot(action) for action in sampled_actions])

        if isinstance(env_model,NNDynamicsModel): # Nawid-  If the env model is a neural net
            env_model_input = env_input_scaler.transform(np.concatenate([m_obs, sampled_actions_one_hot], axis = 1))
            # compute the next state for each sequence
            pred_obs = env_model(torch.tensor(env_model_input).to(device))

            # inverse scaler transformation
            pred_obs = env_output_scaler.inverse_transform(pred_obs.cpu().detach().numpy())
            # add previous observation
            next_obs = pred_obs + m_obs
        else:            
            next_obs = env_model.predict_states(m_obs,sampled_actions) # Nawid - Able to obtain the next state directly rather than predicting a change in states

        if isinstance(rew_model, NNRewardModel):
            rew_model_input = rew_input_scaler.transform(np.concatenate([m_obs, sampled_actions_one_hot, next_obs], axis = 1))
            pred_rew = rew_model(torch.tensor(rew_model_input).to(device)) # Nawid -  I believe I do not need to rescale for a True of false situation
            unroll_rewards +=  (1 - pred_rew.cpu().detach().numpy())
        else:
            pred_rew = rew_model.predict_reward(m_obs, sampled_actions,next_obs)
            unroll_rewards += pred_rew

        m_obs = next_obs # Nawid - Update the state after calculating the new state and calculating the rewards
    
        if t ==0:
            first_sampled_actions = sampled_actions
        
    env_model.train()
    rew_model.train()

    # Best the position of the sequence with the higher reward
    arg_best_reward = np.argmax(unroll_rewards)
    best_sum_reward = unroll_rewards[arg_best_reward].squeeze()
    # take the first action of this sequence
    best_action = first_sampled_actions[arg_best_reward]
    #best_action =  np.squeeze(best_action)
    return best_action, best_sum_reward

# Hyperparameters

In [0]:
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)

# Main loop hyperp
AGGR_ITER = 10
STEPS_PER_AGGR = 500

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 1000

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Supervised Model Hyperp
ENV_LEARNING_RATE = 1e-4
REW_LEARNING_RATE = 1e-4
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  30

# Controller Hyperp
HORIZON_LENGTH = 5
NUM_ACTIONS_SEQUENCES = 200

n_actions = 2
reward_dim = 1

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}.{}.{}".format(now.day, now.hour, now.minute, now.second)

# Decides whether to use a dynamics model or
nn_dynamics = True
nn_reward = True

# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)


In [15]:
project_name = 'CartPole'
wandb.init(entity="nerdk312", project= project_name)

config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.train_model_iter = TRAIN_ITER_MODEL 
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR
config.env_lr = ENV_LEARNING_RATE
config.rew_lr = REW_LEARNING_RATE
config.no_actions = n_actions

# Main

Setting and training of dynamics model

In [16]:
# gather the dataset of random sequences
rand_dataset = gather_random_trajectories(NUM_RAND_TRAJECTORIES,ENV_NAME)
rl_dataset = []
num_examples_added = len(rand_dataset)
env_train, env_val, rew_train, rew_val = collate_data(rand_dataset,rl_dataset,num_examples_added)
norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, norm = normalise_data(env_train, env_val, rew_train,rew_val)

# Initialize the models
if nn_dynamics:
    env_model = NNDynamicsModel(n_actions + env.observation_space.shape[0], env.observation_space.shape[0]).to(device) # Nawid - Need to put both actions as it is a one-hot vector
    env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
    wandb.watch(env_model, log="all")
    env_model_name = 'Env_model'+ '_' + date_time
    early_stopping_env = EarlyStopping(patience=5, verbose=True, wandb=wandb, name=env_model_name)

    for n_iter in range(AGGR_ITER):
        if early_stopping_env.early_stop:
            print('Early stopping')
            break
        train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)
    else:
        env_model = CartPoleTask() # Nawid - Using the oracle for the state

Mean R: 22.28 Max R: 1.0 22.0
len(D): 22281 len(Dtrain) 17825


  7%|▋         | 2/30 [00:00<00:14,  1.98it/s]

Validation loss decreased/improved for Env_model_25_18.33.35  (100000000000.000000 --> 0.232162).  Saving model ...
Validation loss decreased/improved for Env_model_25_18.33.35  (0.232162 --> 0.017513).  Saving model ...


 13%|█▎        | 4/30 [00:01<00:08,  2.98it/s]

Validation loss decreased/improved for Env_model_25_18.33.35  (0.017513 --> 0.010677).  Saving model ...
Validation loss decreased/improved for Env_model_25_18.33.35  (0.010677 --> 0.005642).  Saving model ...


...

EarlyStopping for Env_model_25_18.33.35 counter: 3 out of 5


  7%|▋         | 2/30 [00:00<00:05,  4.95it/s]

EarlyStopping for Env_model_25_18.33.35 counter: 4 out of 5


  7%|▋         | 2/30 [00:00<00:08,  3.20it/s]

EarlyStopping for Env_model_25_18.33.35 counter: 5 out of 5
Env_model_25_18.33.35 has stopped
Early stopping
Early stopping


Setting and Training of reward model

In [17]:
if nn_reward:
    rew_model = NNRewardModel(n_actions + (2*env.observation_space.shape[0]),reward_dim).to(device)
    rew_optimizer = torch.optim.Adam(rew_model.parameters(), REW_LEARNING_RATE)
    wandb.watch(rew_model, log="all")
    desired_model = 'Reward_model'
    model_name = desired_model +'_'+ date_time
    early_stopping_reward = EarlyStopping(patience=5, verbose=True, wandb=wandb, name=model_name)
else:
    rew_model = CartPoleTask()

game_reward = 0

for n_iter in range(AGGR_ITER):
    if early_stopping_reward.early_stop:
        print('Early stopping')
        break
    
    env_train, env_val, rew_train, rew_val = collate_data(rand_dataset,rl_dataset,num_examples_added)# Nawid- Updates the amount of data present
    norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data = normalise_v2(env_train,env_val,rew_train,rew_val,norm)

    #supervised training of the dataset (random and rl if it exists)
    
    train_model(rew_train, rew_val, rew_model, BATCH_SIZE, TRAIN_ITER_MODEL,rew_optimizer,device,early_stopping_reward,desired_model = 'Rew')
    obs = env.reset()

    num_examples_added = 0
    game_reward = 0
    controller_pred_rews = []
    rews = []
    
    while num_examples_added < STEPS_PER_AGGR:
        while True:
            tt = time.time()
            # Execute the control to roll the sequences and pick the first action of the sequence with the higher reward
            action, pred_rew = multi_model_based_control(env_model, rew_model, obs, NUM_ACTIONS_SEQUENCES, HORIZON_LENGTH, env.action_space.sample, norm, device)
            action_one_hot = one_hot(action)
            controller_pred_rews.append(pred_rew)

            # one step in the environment with the action returned by the controller
            new_obs, reward, done, _ = env.step(action)

            # add the last step to the RL dataset
            rl_dataset.append([obs, new_obs, reward, done, action_one_hot])
            num_examples_added += 1
            obs = new_obs
            game_reward += reward

            # if the environment is done, reset it and print some stats
            if done:
                obs = env.reset()
                print('  >> R: {:.2f}, Mean sum:{:.2f}, {}'.format(game_reward, np.mean(controller_pred_rews), num_examples_added))

                rews.append(game_reward)
                game_reward = 0
                controller_pred_rews = []
                break

    print('  >> Mean: {:.2f}', np.mean(rews))

  0%|          | 0/30 [00:00<?, ?it/s]

len(D): 22281 len(Dtrain) 17825


  3%|▎         | 1/30 [00:00<00:15,  1.88it/s]

Validation loss decreased/improved for Reward_model_25_18.33.35  (100000000000.000000 --> 0.604639).  Saving model ...


  7%|▋         | 2/30 [00:00<00:12,  2.29it/s]

...

 43%|████▎     | 13/30 [00:03<00:04,  3.42it/s]

EarlyStopping for Reward_model_25_18.33.35 counter: 4 out of 5


 43%|████▎     | 13/30 [00:04<00:05,  3.16it/s]

EarlyStopping for Reward_model_25_18.33.35 counter: 5 out of 5
Reward_model_25_18.33.35 has stopped
Early stopping


  >> R: 200.00, Mean sum:4.90, 200
  >> R: 200.00, Mean sum:4.94, 400
  >> R: 200.00, Mean sum:4.91, 600
  >> Mean: {:.2f} 200.0
Early stopping


Experimental notes

*   Normalising the target values for the dones seems to make the loss worse, 
*   This is likely due to normalising causes the target values to shift from the 0-1 scale onto a different scale and this can lead to issues when using a binary cross entropy loss which expects values between 0 and 1 (and can lead to a negative loss value)